In [1]:
!pip install pandas
!pip install matplotlib
!pip install scipy

In [2]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.stats import norm

In [3]:
class colors:
    WHITE = '\033[97m'
    BLACK = '\033[90m'
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
def blue(text):
    return colors.BLUE+text+colors.BLACK

def green(text):
    return colors.GREEN+text+colors.BLACK

def red(text):
    return colors.RED+text+colors.BLACK

def yellow(text):
    return colors.YELLOW+text+colors.BLACK

def load_run(runname):
    rundata=pd.read_csv(runname+'.csv')
    
    with open(runname+'_report.json') as f:
        runinfo=json.load(f)

    if 'clock_freq' in runinfo:
        clock_freq=runinfo['clock_freq']
    else:
        clock_freq=None

    if 'clock_cycles' in rundata:
        clock_mean=rundata['clock_cycles'].mean()
        clock_std=rundata['clock_cycles'].std()
        inference_time_mean=clock_mean/(clock_freq) 
    else:
        clock_mean=None
        clock_std=None
        inference_time_mean=None
        
    runinfo["runname"]=runname
    runinfo["rundata"]=rundata
    runinfo["clock_freq"]=clock_freq
    runinfo["clock_mean"]=clock_mean
    runinfo["clock_std"]=clock_std
    runinfo["inference_time_mean"]=inference_time_mean
    
    return runinfo

In [4]:
def consistency(runs, pivot_run):
    # Result consistency, compare to pivot run every other run by mean and std
    for i in range(len(runs)):
        if i!= pivot_run:
            runname=runs[i]["runname"]
            rundata0=abs(runs[i]["rundata"]["probability_0"]-runs[pivot_run]["rundata"]["probability_0"])
            rundata0_mean=rundata0.mean()
            rundata0_std=rundata0.std()
            rundata1=abs(runs[i]["rundata"]["probability_1"]-runs[pivot_run]["rundata"]["probability_1"])
            rundata1_mean=rundata1.mean()
            rundata1_std=rundata1.std()
            prediction=runs[i]["rundata"]["classification"]==runs[pivot_run]["rundata"]["classification"]
           
            print (blue(runname)+" - probability_0 - mean: "+str(rundata0_mean)+ " std: "+str(rundata0_std))
            print (blue(runname)+" - probability_1 - mean: "+str(rundata1_mean)+ " std: "+str(rundata1_std))
            print (blue(runname)+" - prediction    - "+ green(str(prediction.value_counts(normalize=True)[True]*100)+ "%")+" equal")
            print ()

In [5]:
def writetime(runs, pivot_run, x_param, y_param, title):
    x=[]
    y=[]
    names=[]
    for i in range(len(runs)):
        if x_param in runs[i] and y_param in runs[i]:
            x.append(runs[i][x_param])
            y.append(runs[i][y_param])
            names.append(runs[i]["runname"])

            print(runs[i]["runname"], runs[i][x_param], runs[i][y_param])

    if len(names) < 2:
        print ("Not enought data to plot")
        return

In [6]:
def analyze(runs, pivot_run):
    pivot_name=runs[pivot_run]["runname"]
    print ("Pivot run "+  blue(pivot_name))
    print ()
    print (red("Consistency, compare to pivot run every other run"))
    consistency(runs, pivot_run)
    
    print (red("Inference time mean"))
    writetime(runs, pivot_run, "expprec", "inference_time_mean", "")

    print (red("Occupancy"))
    writetime(runs, pivot_run, "cps", "luts", "Occupancy (cps vs luts)")

    print (red("Power consumption"))
    writetime(runs, pivot_run, "cps", "power", "Power consumption (cps vs power)")


In [7]:
soft=load_run("sw")

In [31]:
#expanded=load_run("expanded")
expprec2_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_11bit")
expprec5_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_11bit")
expprec10_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_11bit")
expprec13_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec13_flopoco_11bit")
expprec16_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec16_flopoco_11bit")
expprec20_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_11bit")
print(expprec2_flopoco_11bit.keys())
print('clock_mean' , 'clock_std',\
          'luts', 'power', 'inference_time_mean', 'cps',)
for v in [expprec2_flopoco_11bit,expprec5_flopoco_11bit,\
          expprec10_flopoco_11bit,expprec13_flopoco_11bit,\
          expprec16_flopoco_11bit,expprec20_flopoco_11bit]:
    print(v['clock_mean'] , v['clock_std'], \
          v['luts'], v['power'], v['inference_time_mean'], v['cps'],)

analyze([ soft, expprec2_flopoco_11bit, \
                expprec5_flopoco_11bit, \
                expprec10_flopoco_11bit, \
                expprec13_flopoco_11bit, \
                expprec16_flopoco_11bit, \
                expprec20_flopoco_11bit],0)

dict_keys(['cps', 'bonds', 'luts', 'regs', 'power', 'clock_freq', 'addop', 'divop', 'expprec', 'multop', 'typeprefix', 'runname', 'rundata', 'clock_mean', 'clock_std', 'inference_time_mean'])
clock_mean clock_std luts power inference_time_mean cps
488.9381818181818 213.64094263259784 4708 0.097 4.8893818181818185 24
807.5963636363637 194.26374385486764 4739 0.1 8.075963636363637 24
1378.290909090909 269.9700356938307 4743 0.097 13.78290909090909 24
1289.9418181818182 632.9714568727884 4768 0.098 12.899418181818183 24
964.2254545454546 692.9654461578006 4611 0.093 9.642254545454545 24
895.4727272727273 578.4024138570595 4699 0.097 8.954727272727274 24
Pivot run sw

Consistency, compare to pivot run every other run
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_11bit - probability_0 - mean: 0.022160703163636376 std: 0.02318844332464874
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_11bit - probability_1 - mean: 0.016889607018181815 std: 0.02001580820726566
proj_zedboard_diverge_mm_hwop

Clearly using few bit as soon as we start increasing the series expansion the results explode (we need more bits)
the time simply has a big  uncertainty

In [29]:
#expanded=load_run("expanded")
expprec2_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_16bit")
expprec5_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_16bit")
expprec10_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_16bit")
expprec20_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_16bit")

for v in [expprec2_flopoco_16bit,expprec5_flopoco_16bit,\
          expprec10_flopoco_16bit, expprec20_flopoco_16bit]:
    print(v['clock_mean'] , v['clock_std'], v['cps'], v['luts'], v['power'], v['inference_time_mean'])

analyze([ soft, expprec2_flopoco_16bit, \
                expprec5_flopoco_16bit, \
                expprec10_flopoco_16bit, \
                expprec20_flopoco_16bit],0)

593.5927272727273 1771.2588788783264 24 7733 0.163 5.935927272727273
874.8145454545455 141.77464308787413 24 7716 0.163 8.748145454545455
1504.4072727272728 264.65334981320825 24 7761 0.167 15.044072727272727
2791.9054545454546 542.8027075740333 24 7756 0.162 27.919054545454546
Pivot run sw

Consistency, compare to pivot run every other run
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_16bit - probability_0 - mean: 0.019224126945454532 std: 0.02333690863726429
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_16bit - probability_1 - mean: 0.019153423309090895 std: 0.023290972954059488
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_16bit - prediction    - 100.0% equal

proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_16bit - probability_0 - mean: 0.0012532669454545485 std: 0.0011708291299422364
proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_16bit - probability_1 - mean: 0.0011948972000000008 std: 0.0012303044035162469
proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_16bit - prediction  

Clearly using few bit as soon as we start increasing the series expansion the results explode (we need more bits)
as for 11 but ony for 20 

In [23]:
#expanded=load_run("expanded")
expprec2_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_19bit")
expprec5_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_19bit")
expprec10_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_19bit")
expprec13_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec13_flopoco_19bit")
expprec16_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec16_flopoco_19bit")
expprec20_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_19bit")

for v in [expprec2_flopoco_19bit,expprec5_flopoco_19bit,\
          expprec10_flopoco_19bit,expprec13_flopoco_19bit,\
          expprec16_flopoco_19bit,expprec20_flopoco_19bit]:
    print(v['clock_mean'] , v['clock_std'] )

analyze([ soft, expprec2_flopoco_19bit, \
                expprec5_flopoco_19bit, \
                expprec10_flopoco_19bit, \
                expprec13_flopoco_19bit, \
                expprec16_flopoco_19bit, \
                expprec20_flopoco_19bit],0)

504.40363636363634 70.01715630122105
921.4727272727273 152.11684600143846
1595.7854545454545 271.9753795602237
2017.7854545454545 363.54775759235713
2370.1963636363635 440.75179385040406
2967.2618181818184 511.3580071662442
Pivot run sw

Consistency, compare to pivot run every other run
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_19bit - probability_0 - mean: 0.01939879639999999 std: 0.023259414086205046
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_19bit - probability_1 - mean: 0.01939840054545453 std: 0.02325143079667957
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_19bit - prediction    - 100.0% equal

proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_19bit - probability_0 - mean: 0.0002485682545454503 std: 0.00041605595958265344
proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_19bit - probability_1 - mean: 0.0002516501454545457 std: 0.0004182685867528347
proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_19bit - prediction    - 100.0% equal

proj_zedboard_diverge_mm_hwopt_expprec

In [24]:
#expanded=load_run("expanded")
expprec2_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_32bit")
expprec5_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_32bit")
expprec10_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_32bit")
expprec13_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec13_flopoco_32bit")
expprec16_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec16_flopoco_32bit")
expprec20_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_32bit")

for v in [expprec2_flopoco_32bit,expprec5_flopoco_32bit,\
          expprec10_flopoco_32bit,expprec13_flopoco_32bit,\
          expprec16_flopoco_32bit,expprec20_flopoco_32bit]:
    print(v['clock_mean'] , v['clock_std'] )

analyze([ soft, expprec2_flopoco_32bit, \
                expprec5_flopoco_32bit, \
                expprec10_flopoco_32bit, \
                expprec13_flopoco_32bit, \
                expprec16_flopoco_32bit, \
                expprec20_flopoco_32bit],0)

547.709090909091 73.85655675326268
990.5127272727273 153.84553236973144
1679.4472727272728 303.7008347489456
2207.141818181818 392.4100690552579
2625.4145454545455 456.5234171919245
3118.690909090909 562.463010064463
Pivot run sw

Consistency, compare to pivot run every other run
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_32bit - probability_0 - mean: 0.019376391272727275 std: 0.023257574340262332
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_32bit - probability_1 - mean: 0.019376387672727285 std: 0.023257570784687022
proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_32bit - prediction    - 100.0% equal

proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_32bit - probability_0 - mean: 0.00019475625454545742 std: 0.00042261806254662834
proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_32bit - probability_1 - mean: 0.00019476709090909616 std: 0.0004226243074933183
proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_32bit - prediction    - 100.0% equal

proj_zedboard_diverge_mm_hwopt_expprec10